In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./distilgpt2-wekeza-finetuned_v3_lora"

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D(nf=2304, nx=768)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): lora.Linear(
            (base_layer): Conv1D(nf=768, nx=768)
            (l

In [16]:
wekeza_constitution = [
    "Always give investment advice that is realistic and tailored to Kenya.",
    "Do not promise guaranteed returns.",
    "If asked about risk, explain that all investments carry some form of risk.",
    "Avoid giving financial advice that promotes scams, betting, or unregulated platforms.",
    "Always recommend verified, regulated entities (e.g., CMA licensed MMFs).",
    "If unsure about a fact, say so rather than guessing.",
    "Be concise, respectful, and informative."
]


In [23]:
#initial response
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [24]:
#critique the response using the constitution
def critique_response(prompt, response, constitution):
    critique_prompt = f"""You are an AI trained to follow ethical investment guidelines in Kenya. Below is a user question and the assistant's answer.

Your task is to write a short critique pointing out any harmful, unrealistic, repetitive, or vague advice in the assistant's answer, using the constitution.

Constitution:
{constitution}

User Question:
{prompt}

Assistant's Answer:
{response}

Critique:"""
    return generate_response(critique_prompt, max_new_tokens=150)



In [25]:
#revising the response using the critique
def revise_response(prompt, response, critique):
    revision_prompt = f"""You are a helpful Kenyan financial assistant. Improve the assistant’s original answer using the critique. The revised answer should be helpful, realistic, and non-repetitive.

User Question:
{prompt}

Original Answer:
{response}

Critique:
{critique}

Improved Answer:"""
    return generate_response(revision_prompt, max_new_tokens=150)



In [26]:
#dataset formatting
import json

def save_aligned_example(prompt, original, critique, revised, path="wekeza_self_aligned.jsonl"):
    entry = {
        "instruction": f"{prompt}\n\nOriginal: {original}\n\nCritique: {critique}",
        "input": "",
        "output": revised
    }
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")


In [27]:
#full loop automation
prompt = "Which is the best money market fund in Kenya right now?"

original = generate_response(prompt)
critique = critique_response(prompt, original, "\n".join(wekeza_constitution))
revised = revise_response(prompt, original, critique)

print("Original:\n", original)
print("\nCritique:\n", critique)
print("\nRevised:\n", revised)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Original:
 Which is the best money market fund in Kenya right now?

### Response:
Yes, the fund offers excellent returns, with returns averaging 8-11% annually, with average returns averaging 9-11%. However, if you're in Kenya, you'll need to invest in several different fund managers to get the most out of your money. Check with your bank or bank to make sure you're aware of their policies and regulations. Check your bank's website for more information. Check their website for additional information.
### Return:
Compare returns vs

Critique:
 You are an AI trained to follow ethical investment guidelines in Kenya. Below is a user question and the assistant's answer.

Your task is to write a short critique pointing out any harmful, unrealistic, repetitive, or vague advice in the assistant's answer, using the constitution.

Constitution:
Always give investment advice that is realistic and tailored to Kenya.
Do not promise guaranteed returns.
If asked about risk, explain that all investmen

In [22]:
print("Original:\n", original)
print("\nCritique:\n", critique)
print("\nRevised:\n", revised)


Original:
 Which is the best money market fund in Kenya right now?

### Response:
Yes, the fund offers excellent returns, with returns averaging 8-11% annually, with returns averaging 8-11% annually. However, if you're in Kenya, you'll need to invest in several different fund managers to get the most out of your investment. Check with your local bank to make sure you're aware of their investment options. Check with your local bank to make sure you're aware of their investment options. Check with your local bank to make sure you

Critique:
 
Given the following user prompt and assistant response, critique the assistant’s response based on the following principles:

Constitution:
Always give investment advice that is realistic and tailored to Kenya.
Do not promise guaranteed returns.
If asked about risk, explain that all investments carry some form of risk.
Avoid giving financial advice that promotes scams, betting, or unregulated platforms.
Always recommend verified, regulated entities 

In [30]:
import json
import os
from datetime import datetime

alignment_data_file = "constitutionaligned_kenyan_finance_dataset_v3.jsonl"
backup_dir = "aligned_backups"
os.makedirs(backup_dir, exist_ok=True)

def save_aligned_example(prompt, original, critique, revised):
    data = {
        "prompt": prompt,
        "original": original,
        "critique": critique,
        "revised": revised
    }

    # Save to main aligned dataset
    with open(alignment_data_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

    # Also save timestamped backup in subfolder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_path = os.path.join(backup_dir, f"{timestamp}.jsonl")
    with open(backup_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

    print(f"Example saved to {alignment_data_file} and backup created at {backup_path}")


In [31]:
prompt = "Explain the risks of investing in unregulated SACCOs in Kenya."

original = generate_response(prompt)
critique = critique_response(prompt, original, "\n".join(wekeza_constitution))
revised = revise_response(prompt, original, critique)

save_aligned_example(prompt, original, critique, revised)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Example saved to constitutionaligned_kenyan_finance_dataset_v3.jsonl and backup created at aligned_backups\20250801_154517.jsonl


In [35]:
!pip install -q peft transformers datasets accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import json


In [37]:
my_model_path = "./distilgpt2-wekeza-finetuned_v3_lora"

tokenizer = AutoTokenizer.from_pretrained(my_model_path)
model = AutoModelForCausalLM.from_pretrained(my_model_path)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [38]:
def load_jsonl(file_path):
    with open(file_path, 'r') as f:
        lines = [json.loads(line) for line in f]
    return Dataset.from_list(lines)

dataset = load_jsonl("constitutionaligned_kenyan_finance_dataset_v3.jsonl")


In [39]:
#formatting the dataset for finetuning
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['prompt']}\n\n### Response:\n{example['revised']}"
    }

dataset = dataset.map(format_example)


Map: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 163.71 examples/s]


In [40]:
#tokenizing
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 75.01 examples/s]


In [41]:
#lora
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # for distilgpt2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 405,504 || all params: 82,318,080 || trainable%: 0.4926


C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\peft\tuners\tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [42]:
training_args = TrainingArguments(
    output_dir="./distilgpt2-wekeza-finetuned_v4_lora",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


C:\Users\bbollo\AppData\Local\Temp\ipykernel_16176\1889173795.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3, training_loss=2.9846413930257163, metrics={'train_runtime': 5.9333, 'train_samples_per_second': 0.506, 'train_steps_per_second': 0.506, 'total_flos': 395682250752.0, 'train_loss': 2.9846413930257163, 'epoch': 3.0})

In [44]:
trainer.save_model("./distilgpt2-wekeza-finetuned_v4_lora")
tokenizer.save_pretrained("./distilgpt2-wekeza-finetuned_v4_lora")


('./distilgpt2-wekeza-finetuned_v4_lora\\tokenizer_config.json',
 './distilgpt2-wekeza-finetuned_v4_lora\\special_tokens_map.json',
 './distilgpt2-wekeza-finetuned_v4_lora\\vocab.json',
 './distilgpt2-wekeza-finetuned_v4_lora\\merges.txt',
 './distilgpt2-wekeza-finetuned_v4_lora\\added_tokens.json',
 './distilgpt2-wekeza-finetuned_v4_lora\\tokenizer.json')